# COBRApyの使い方

In [1]:
import numpy as np
import pandas as pd
import cobra
from cobra import Model, Reaction, Metabolite

## 設定を確認/変更
* 使う線形計画法ソルバー


In [2]:
cobra_config = cobra.Configuration()
cobra_config

Attribute,Description,Value
solver,Mathematical optimization solver,glpk
tolerance,"General solver tolerance (feasibility, integrality, etc.)",1e-07
lower_bound,Default reaction lower bound,-1000.0
upper_bound,Default reaction upper bound,1000.0
processes,Number of parallel processes,7
cache_directory,Path for the model cache,/Users/maruyamatooru/Library/Caches/cobrapy
max_cache_size,Maximum cache size in bytes,104857600
cache_expiration,Model cache expiration time in seconds (if any),None


## モデルを設計する

実際にはRecon3DやAGORAのような既存のモデルを使うことが多いと思うが、ゼロからモデルを設計する場合は以下のように実行する。

### Declare `Model` class

In [4]:
model = Model('example_model')
model

Name,example_model
Memory address,0x07f805264a490
Number of metabolites,0
Number of reactions,0
Number of groups,0
Objective expression,0
Compartments,


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


### Declare `Reaction` class

In [10]:
reaction = Reaction('R_3OAS140')
reaction.name = '3 oxoacyl acyl carrier protein synthase n C140 '
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [11]:
reaction

Reaction identifier,R_3OAS140
Name,3 oxoacyl acyl carrier protein synthase n C140
Memory address,0x07f8057550e50
Stoichiometry,--> -->
GPR,
Lower bound,0.0
Upper bound,1000.0


### Declare `Metabolite` class

* compartment
    * c: 細胞内
    * e: 細胞外

In [13]:
ACP_c = Metabolite('ACP_c', formula='C11H21N2O7PRS', name='acyl-carrier-protein', compartment='c')
omrsACP_c = Metabolite('M3omrsACP_c', formula='C25H45N2O9PRS', name='3-Oxotetradecanoyl-acyl-carrier-protein', compartment='c')
co2_c = Metabolite('co2_c', formula='CO2', name='CO2', compartment='c')
malACP_c = Metabolite('malACP_c', formula='C14H22N2O10PRS', name='Malonyl-acyl-carrier-protein', compartment='c')
h_c = Metabolite('h_c', formula='H', name='H', compartment='c')
ddcaACP_c = Metabolite('ddcaACP_c', formula='C23H43N2O8PRS', name='Dodecanoyl-ACP-n-C120ACP', compartment='c')

In [14]:
ACP_c

Metabolite identifier,ACP_c
Name,acyl-carrier-protein
Memory address,0x07f805810f310
Formula,C11H21N2O7PRS
Compartment,c
In 0 reaction(s),


### Add stoichiometry in `Reaction` object

In [15]:
reaction.add_metabolites({
    malACP_c: -1.0,
    h_c: -1.0,
    ddcaACP_c: -1.0,
    co2_c: 1.0,
    ACP_c: 1.0,
    omrsACP_c: 1.0
})
reaction

Reaction identifier,R_3OAS140
Name,3 oxoacyl acyl carrier protein synthase n C140
Memory address,0x07f8057550e50
Stoichiometry,ddcaACP_c + h_c + malACP_c --> ACP_c + M3omrsACP_c + co2_c Dodecanoyl-ACP-n-C120ACP + H + Malonyl-acyl-carrier-protein --> acyl-carrier-protein + 3-Oxotetradecanoyl-acyl-carrier-protein + CO2
GPR,
Lower bound,0.0
Upper bound,1000.0


#### `Reaction.metabolites`

In [21]:
reaction.metabolites

{<Metabolite malACP_c at 0x7f805810f2e0>: -1.0,
 <Metabolite h_c at 0x7f805810f370>: -1.0,
 <Metabolite ddcaACP_c at 0x7f805810f3a0>: -1.0,
 <Metabolite co2_c at 0x7f805810f340>: 1.0,
 <Metabolite ACP_c at 0x7f805810f310>: 1.0,
 <Metabolite M3omrsACP_c at 0x7f805810f2b0>: 1.0}

In [22]:
reaction.metabolites[malACP_c] # `Metabolite` object is the key

-1.0

#### `Reaction.reaction`

化学量論式をstring型で格納

In [25]:
reaction.reaction

'ddcaACP_c + h_c + malACP_c --> ACP_c + M3omrsACP_c + co2_c'

#### `Reaction.genes`
代謝を担う遺伝子 `Reaction.gene_reaction_rule`で指定する。遺伝子名の記法については[Schellenberger et al. 2011](http://dx.doi.org/doi:10.1038/nprot.2011.308)を参照。

In [26]:
reaction.gene_reaction_rule = '( STM2378 or STM1197 )'
reaction.genes

frozenset({<Gene STM1197 at 0x7f805810fe80>, <Gene STM2378 at 0x7f805810fe20>})

### Add reaction in `Model` object

In [27]:
model.add_reactions([reaction])

#### `Model.reactions`

In [28]:
model.reactions

[<Reaction R_3OAS140 at 0x7f8057550e50>]

#### `Model.metabolites`

In [30]:
model.metabolites

[<Metabolite malACP_c at 0x7f805810f2e0>,
 <Metabolite h_c at 0x7f805810f370>,
 <Metabolite ddcaACP_c at 0x7f805810f3a0>,
 <Metabolite co2_c at 0x7f805810f340>,
 <Metabolite ACP_c at 0x7f805810f310>,
 <Metabolite M3omrsACP_c at 0x7f805810f2b0>]

### その他の反応

#### `Model.exchanges`
* 細胞外の物質の消費・追加を表現する場合
* 可逆的

In [41]:
model.exchanges

There are no boundary reactions in this model. Therefore, specific types of boundary reactions such as 'exchanges', 'demands' or 'sinks' cannot be identified.


[]

#### `Model.demands`
* 細胞内の物質の消費を表現する場合
* 不可逆

In [42]:
model.demands

There are no boundary reactions in this model. Therefore, specific types of boundary reactions such as 'exchanges', 'demands' or 'sinks' cannot be identified.


[]

#### `Model.sinks`
* 細胞内の物質の消費・追加を表現する場合
* 可逆的

In [43]:
model.sinks

There are no boundary reactions in this model. Therefore, specific types of boundary reactions such as 'exchanges', 'demands' or 'sinks' cannot be identified.


[]

### 目的関数の設定

In [37]:
model.objective = 'R_3OAS140'

In [40]:
model.objective.direction

'max'

## モデルの読み込み

In [44]:
import cobra.test
data_dir = cobra.test.data_dir
data_dir

'/Users/maruyamatooru/opt/anaconda3/envs/cobra/lib/python3.9/site-packages/cobra/test/data/'

### SBML
SBMLとはSystems Biology Markdown Languageのことで代謝やシグナルパスウェイを記述する用途でよく用いられるフォーマット

In [46]:
cobra.io.read_sbml_model(os.path.join(data_dir, "mini_fbc2.xml"))

Name,mini_textbook
Memory address,0x07f805810fb20
Number of metabolites,23
Number of reactions,18
Number of groups,0
Objective expression,1.0*ATPM - 1.0*ATPM_reverse_5b752 + 1.0*PFK - 1.0*PFK_reverse_d24a6
Compartments,"cytosol, extracellular"


### JSON

In [48]:
cobra.io.load_json_model(os.path.join(data_dir, "mini.json"))

Name,mini_textbook
Memory address,0x07f805814d100
Number of metabolites,23
Number of reactions,18
Number of groups,0
Objective expression,1.0*ATPM - 1.0*ATPM_reverse_5b752 + 1.0*PFK - 1.0*PFK_reverse_d24a6
Compartments,"cytosol, extracellular"


### YAML

In [49]:
cobra.io.load_yaml_model(os.path.join(data_dir, "mini.yml"))

Name,mini_textbook
Memory address,0x07f8058568130
Number of metabolites,23
Number of reactions,18
Number of groups,0
Objective expression,1.0*ATPM - 1.0*ATPM_reverse_5b752 + 1.0*PFK - 1.0*PFK_reverse_d24a6
Compartments,"cytosol, extracellular"


## Flux Balance Analysis (FBA)

In [50]:
model = cobra.test.create_test_model("textbook") # test model
model

Name,e_coli_core
Memory address,0x07f80582cd7f0
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba
Compartments,"cytosol, extracellular"


In [51]:
model.objective.direction

'max'

In [62]:
model.reactions.get_by_id("Biomass_Ecoli_core").reaction

'1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c'

以下の関数`Model.optimize()`で上記の目的関数 `1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba` を最大化する最適化問題を解く。

In [52]:
solution = model.optimize()
print(solution)

<Solution 0.874 at 0x7f80582cd610>


* `solution.objective_value`: 最適化後の目的関数の値
* `solution.status`: 
* `solution.fluxes`: 最適化後の各反応のFlux `pandas.Series`型
* `solution.shadow_prices`: shadow price (詳細は[Yugiさんの資料](http://kurodalab.bs.s.u-tokyo.ac.jp/member/Yugi/Textbook/chapter5_slide.pdf)). 0=当該物質の流入・流出が目的関数の変化に寄与しない

In [53]:
solution.objective_value

0.8739215069684307

In [54]:
solution.status

'optimal'

In [55]:
solution.fluxes

ACALD     0.000000
ACALDt    0.000000
ACKr      0.000000
ACONTa    6.007250
ACONTb    6.007250
            ...   
TALA      1.496984
THD2      0.000000
TKT1      1.496984
TKT2      1.181498
TPI       7.477382
Name: fluxes, Length: 95, dtype: float64

In [58]:
solution.shadow_prices.sort_values()

s7p_c     -0.113308
fdp_c     -0.104396
g6p_c     -0.098030
f6p_c     -0.098030
6pgc_c    -0.091665
             ...   
nad_c      0.007639
nadp_c     0.008912
amp_c      0.010185
accoa_c    0.026736
coa_c      0.054744
Name: shadow_prices, Length: 72, dtype: float64

In [63]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.765,0,0.00%
o2_e,EX_o2_e,21.8,0,0.00%
pi_e,EX_pi_e,3.215,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-22.81,1,100.00%
h2o_e,EX_h2o_e,-29.18,0,0.00%
h_e,EX_h_e,-17.53,0,0.00%


In [64]:
model.metabolites.nadh_c.summary()

Percent,Flux,Reaction,Definition
13.14%,5.064,AKGDH,akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c
8.04%,3.1,Biomass_Ecoli_core,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c
41.58%,16.02,GAPD,g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
13.14%,5.064,MDH,mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
24.09%,9.283,PDH,coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
Percent,Flux,Reaction,Definition
100.00%,-38.53,NADH16,4.0 h_c + nadh_c + q8_c --> 3.0 h_e + nad_c + q8h2_c


目的関数を変える場合は以下のように

In [65]:
model.objective = "ATPM"

In [66]:
model

Name,e_coli_core
Memory address,0x07f80582cd7f0
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*ATPM - 1.0*ATPM_reverse_5b752
Compartments,"cytosol, extracellular"


## 遺伝子欠損の影響をシミュレーション

In [76]:
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

cobra_model = cobra.test.create_test_model("textbook")
ecoli_model = cobra.test.create_test_model("ecoli")
cobra_model

Name,e_coli_core
Memory address,0x07f80408a0280
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba
Compartments,"cytosol, extracellular"


### Wild type

In [77]:
cobra_model.optimize().objective_value

0.8739215069684307

### 特定の代謝経路をノックアウトした場合

In [84]:
# phostphofructokinase
cobra_model.reactions.PFK

Reaction identifier,PFK
Name,phosphofructokinase
Memory address,0x07f803fd654f0
Stoichiometry,"atp_c + f6p_c --> adp_c + fdp_c + h_c ATP + D-Fructose 6-phosphate --> ADP + D-Fructose 1,6-bisphosphate + H+"
GPR,b3916 or b1723
Lower bound,0.0
Upper bound,1000.0


In [82]:
with cobra_model:
    cobra_model.reactions.PFK.knock_out()
    print(cobra_model.optimize().objective_value)

0.7040369478590214


### 特定の遺伝子をノックアウトした場合

In [85]:
with cobra_model:
    cobra_model.genes.b1723.knock_out()
    print(cobra_model.optimize().objective_value)
    cobra_model.genes.b3916.knock_out()
    print(cobra_model.optimize().objective_value) # phosphofructokinaseをコードする2つの遺伝子がどちらも欠損した場合のみ影響

0.8739215069684295
0.7040369478590242


### 全代謝反応/遺伝子のKOの影響を計算可能

In [86]:
deletion_results = single_gene_deletion(cobra_model)
deletion_results.head()

,ids,growth,status
0,{b3962},0.873922,optimal
1,{b0724},0.814298,optimal
2,{b1854},0.873922,optimal
3,{b0451},0.873922,optimal
4,{b0114},0.796696,optimal


In [91]:
single_reaction_deletion(cobra_model, cobra_model.reactions[:20])

,ids,growth,status
0,{AKGDH},8.583074e-01,optimal
1,{ACKr},8.739215e-01,optimal
2,{Biomass_Ecoli_core},0.000000e+00,optimal
3,{ACALD},8.739215e-01,optimal
4,{ACONTa},2.602325e-15,optimal
5,{ETOHt2r},8.739215e-01,optimal
6,{CS},2.602325e-15,optimal
7,{D_LACt2},8.739215e-01,optimal
8,{ACt2r},8.739215e-01,optimal
9,{AKGt2r},8.739215e-01,optimal


In [97]:
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

In [110]:
model.reactions.get_by_id(list(model.medium.keys())[0])

Reaction identifier,EX_co2_e
Name,CO2 exchange
Memory address,0x07f8058471bb0
Stoichiometry,co2_e <=> CO2 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [112]:
model.exchanges.get_by_id(list(model.medium.keys())[0])

Reaction identifier,EX_co2_e
Name,CO2 exchange
Memory address,0x07f8058471bb0
Stoichiometry,co2_e <=> CO2 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


# Recon3D

In [119]:
recon3d = cobra.io.read_sbml_model("Recon3D.xml")
recon3d

Name,Recon3D
Memory address,0x07f8043805100
Number of metabolites,5835
Number of reactions,10600
Number of groups,0
Objective expression,1.0*BIOMASS_maintenance - 1.0*BIOMASS_maintenance_reverse_5b3f9
Compartments,"cytosol, lysosome, mitochondria, endoplasmic reticulum, extracellular space, peroxisome/glyoxysome, nucleus, golgi apparatus, inner mitochondrial compartment"


In [124]:
len(recon3d.exchanges), len(recon3d.medium)

(1560, 1560)

In [125]:
op = recon3d.optimize()

In [133]:
op.fluxes[[x.id for x in recon3d.exchanges]].sort_values()

EX_cbasp_e     -1000.0
EX_cmp_e       -1000.0
EX_HC00250_e   -1000.0
EX_thymd_e     -1000.0
EX_alltn_e     -1000.0
                 ...  
EX_dttp_e       1000.0
EX_succ_e       1000.0
EX_ctp_e        1000.0
EX_mi1p__D_e    1000.0
EX_adpoh_e      1000.0
Name: fluxes, Length: 1560, dtype: float64

In [134]:
op.objective_value

755.0032155506631

In [150]:
recon3d.exchanges[0].metabolites

{<Metabolite 5adtststerone_e at 0x7f803bc2e460>: -1.0}

In [153]:
recon3d.exchanges.EX_h2o_e

Reaction identifier,EX_h2o_e
Name,H2O exchange
Memory address,0x07f802fe8ab20
Stoichiometry,h2o_e <=> H2O H2O <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0
